In [1]:
import numpy as np
import tensorflow as tf

import sys
sys.path.append(r'D:\Users\Madhouse\tensorflow\yobiface\src')
import utils

import os
from os.path import join as pjoin
import copy
import shutil
import pickle
import time as t
 
import cv2  
import detect_face  
import time 
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

d:\Users\Madhouse\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.7.0


In [17]:
def load_all_pics(path):
    data = {}
    pics_ctr = 0
    files= os.listdir(path)
    for file in files: #遍历文件夹 
        if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
        #print(file)
            person_dir = pjoin(path, file)
            if(file.find(".jpg") > 0):
                guy=file.split("/")[-1][:-4]
                #print("guy[%d]=%s"%(pics_ctr,guy))
                curr_pics = [utils.preproc(cv2.imread(person_dir))]
                data[guy] = curr_pics
                pics_ctr += len(curr_pics)
    return data, pics_ctr

def load_emb(path):
    pickle_file = open(path, 'rb')
    my_dict = pickle.load(pickle_file)
    # 一定要注意 要写关闭文件
    pickle_file.close()
    print("read dict from file is finished")
    return my_dict

def getTime():
    return t.strftime('%Y-%m-%d %H:%M:%S',t.localtime())

def weigth_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], padding='SAME')

def conv1d(x, W):
    return tf.nn.conv1d(x, W, stride=1, padding='SAME')
def max_pool_1d(x):
    return tf.layers.max_pooling1d(x, pool_size=[2],strides=[2], padding='SAME')

def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = weigth_variable([input_size, size])
    b = bias_variable([size])
    return tf.matmul(input_layer, W) +b

def inferenceCNN(images, keep_prob, nbr_class):
    W_conv1 = weigth_variable([5, 1, 32])
    b_conv1 = bias_variable([32])
        
    x_image = tf.reshape(images, [-1, 512, 1])
        
    h_conv1 = tf.nn.relu(conv1d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_1d(h_conv1)
    
    W_conv2 = weigth_variable([5, 32, 64])
    b_conv2 = bias_variable([64])
    
    h_conv2 = tf.nn.relu(conv1d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_1d(h_conv2)
    
    #W_fc1 = weigth_variable([64, 1024])
    #b_fc1 = bias_variable([1024])
    
    #h_pool2_flat = tf.reshape(h_pool2, [-1, 64])
    #h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    #h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    #W_fc2 = weigth_variable([1024, nbr_class])
    #b_fc2 = bias_variable([nbr_class])
    #y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    
    flat = tf.reshape(h_pool2,[-1,h_pool2.get_shape()[1]*h_pool2.get_shape()[2]])

    fully_conected = tf.nn.relu(normal_full_layer(flat,1024))
    
    second_hidden_layer = tf.nn.relu(normal_full_layer(fully_conected,512))

    full_one_dropout = tf.nn.dropout(second_hidden_layer,keep_prob=keep_prob)

    y_pred = normal_full_layer(full_one_dropout,nbr_class)
    pred_softmax = tf.nn.softmax(y_pred)

    return pred_softmax

def loss(logits, labels):
    """Calculates the loss from the logits and the labels.

    Args:
      logits: Logits tensor, float - [batch_size, NUM_CLASSES].
      labels: Labels tensor, int32 - [batch_size].

    Returns:
      loss: Loss tensor of type float.
    """
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels, name='xentropy')
    # softmax_cross_entropy_with_logits
    lo = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return lo

def getBatch1(arr_x, arr_y, iter_cnt, batch_size=100):
    #assert (Temperature >= 0),"Colder than absolute zero!"
    assert(len(arr_x)==len(arr_y)),"lens of arr_x and arr_y are not equal..."
    length=len(arr_x)
    maxcnt=length//batch_size + 1
    iter_cnt=iter_cnt%maxcnt
    start=batch_size*(iter_cnt)
    end=min(batch_size*(iter_cnt+1),length)
    return arr_x[start:end],arr_y[start:end]

def getBatch(data_set, iter_cnt, batch_size=100):
    length=len(data_set)
    maxcnt=length//batch_size + 1
    iter_cnt=iter_cnt%maxcnt
    start=batch_size*(iter_cnt)
    end=min(batch_size*(iter_cnt+1),length)
    return data_set[start:end][:,1:],data_set[start:end][:,0]

def fill_feed_dictCNN(data_set, type, images_pl, labels_pl, keep_prob, iter_cnt, batch_size=100):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
        <placeholder>: <tensor of values to be passed for placeholder>,
        ....
    }
    Args:
      data_set: The set of images and labels, from input_data.read_data_sets()
      images_pl: The images placeholder, from placeholder_inputs().
      labels_pl: The labels placeholder, from placeholder_inputs().
    Returns:
      feed_dict: The feed dictionary mapping from placeholders to values.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    #batch = data_set.next_batch(FLAGS.batch_size, FLAGS.fake_data)
    x,y=getBatch(data_set, iter_cnt, batch_size)
    kp = 0.5
    if type.find("train") == -1:
        kp = 1.0 
    feed_dict = {
        images_pl: x,
        labels_pl: y,
      keep_prob: kp
    }
    return feed_dict

In [3]:
emb_arr = load_emb('emb_arr.pkl')
print('emb_arr length = %d'%len(emb_arr))
emb_map = load_emb('emb_map.pkl')
print('emb_map length = %d'%len(emb_map))

x,y=getBatch(emb_arr, 0, 500)
print(x.shape, y.shape)

read dict from file is finished
emb_arr length = 13268
read dict from file is finished
emb_map length = 5761
(500, 512) (500,)


In [4]:
#face detection parameters  
minsize = 20 # minimum size of face  
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold  
factor = 0.709 # scale factor 
print('Creating networks and loading parameters')  
with tf.Graph().as_default():  
    sess = tf.Session() 
    with sess.as_default():  
        pnet, rnet, onet = detect_face.create_mtcnn(sess, './mtcnn_model/')
        saver = tf.train.import_meta_graph(r'D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.meta')
        saver.restore(sess, r'D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.ckpt-275')
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
print("model restore finished.....")

Creating networks and loading parameters
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.ckpt-275
model restore finished.....


In [ ]:
class_nbr=len(emb_map)
x= tf.placeholder("float", shape=[None, 512])
#y_ = tf.placeholder(tf.int32, shape=[None], name="y")
y_ = tf.placeholder("float", shape=[None], name="y")
#y = tf.placeholder(tf.int32, shape=[None], name="y")
keep_prob = tf.placeholder("float")

logits=inferenceCNN(x, keep_prob, class_nbr)
print(logits.shape)
lo = loss(logits, y_)
optimizer = tf.train.GradientDescentOptimizer(1e-4)
train_op = optimizer.minimize(lo)
#labels = tf.to_int64(labels)
cp = tf.nn.in_top_k(logits, tf.to_int32(y_), 1)
correct=tf.reduce_sum(tf.cast(cp, tf.int32))

se = tf.Session()
se.run(tf.global_variables_initializer())
with se.as_default():
    for i in range(50000):
        batch_size=500
        feed_dict=fill_feed_dictCNN(emb_arr, "train", x, y_, keep_prob, i, batch_size)
        #batch=getBatch(emb_arr, i, batch_size)
        kp = 0.5
        tt = getTime()
        if i%200 == 0:
            train_accuracy = correct.eval(feed_dict=feed_dict)
            print("[%s] step %d, training accuracy %g"%(tt, i, train_accuracy))
        train_op.run(feed_dict=feed_dict)
        #train_op.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: kp})

(?, 5761)
[2018-06-27 09:33:24] step 0, training accuracy 1%
[2018-06-27 09:36:41] step 200, training accuracy 0%
[2018-06-27 09:39:55] step 400, training accuracy 0%
[2018-06-27 09:43:06] step 600, training accuracy 0%
[2018-06-27 09:46:21] step 800, training accuracy 0%
[2018-06-27 09:49:35] step 1000, training accuracy 0%
[2018-06-27 09:52:50] step 1200, training accuracy 0%
[2018-06-27 09:56:32] step 1400, training accuracy 0%
[2018-06-27 10:00:12] step 1600, training accuracy 0%
[2018-06-27 10:03:28] step 1800, training accuracy 0%
[2018-06-27 10:06:52] step 2000, training accuracy 0%
[2018-06-27 10:10:18] step 2200, training accuracy 0%
[2018-06-27 10:13:38] step 2400, training accuracy 0%
[2018-06-27 10:17:02] step 2600, training accuracy 0%
[2018-06-27 10:20:24] step 2800, training accuracy 0%
[2018-06-27 10:23:53] step 3000, training accuracy 0%
[2018-06-27 10:27:30] step 3200, training accuracy 1%
[2018-06-27 10:31:04] step 3400, training accuracy 0%
[2018-06-27 10:34:33] ste